## ACTIVIDAD EN CLASE ##
### PROCESAMIENTO Y LIMPIEZA DE DATOS DE CLIENTES EN ARCHIVOS TXT, CSV, JSON Y XML UTILIZANDO PYTHON ###

- 1. Lectura y limpieza de clientes.txt
- leer el archivo linea por linea ignorando lineas vacias o con espacios
- limpiar cada nombre tuilizando strip() y title ()
- Almacenar los nombres limpios en una lista y registrar en el log cuantos nombres unicos se obtuvieron.

In [1]:
nombres_limpios = set()

with open('clientes.txt', 'r', encoding='utf-8') as archivo:
    for linea in archivo:
        nombre = linea.strip()
        if nombre:  
            nombre_limpio = nombre.title()
            nombres_limpios.add(nombre_limpio)

with open('log.txt', 'a', encoding='utf-8') as log:
    log.write(f'Se obtuvieron {len(nombres_limpios)} nombres únicos.\n')

2. Lectura de parámetros desde config.json
- Leer los parámetros "limite_ventas" y "ordenar_por" desde el archivo JSON.
- Imprimir estos valores como parte del log del script.

In [2]:
import json

with open('config.json', 'r', encoding='utf-8') as config_file:
    config = json.load(config_file)
    limite_ventas = config.get('limite_ventas')
    ordenar_por = config.get('ordenar_por')

with open('log.txt', 'a', encoding='utf-8') as log:
    log.write(f'Parámetros leídos: limite_ventas={limite_ventas}, ordenar_por={ordenar_por}\n')

3. Procesamiento de ventas.csv con pandas
- Leer el archivo CSV con pandas.
- Filtrar ventas mayores o iguales al valor definido en "limite_ventas".
- Agrupar por cliente y calcular el total de ventas por nombre.
- Ordenar según el campo "ordenar_por".

In [3]:
import pandas as pd

ventas_df = pd.read_csv('ventas.csv')

ventas_filtradas = ventas_df[ventas_df['Monto'] >= limite_ventas]

ventas_agrupadas = ventas_filtradas.groupby('Nombre', as_index=False)['Monto'].sum()

ventas_agrupadas = ventas_agrupadas.rename(columns={'Monto': 'monto_total', 'Nombre': 'cliente'})

ventas_ordenadas = ventas_agrupadas.sort_values(by=ordenar_por, ascending=False)

with open('log.txt', 'a', encoding='utf-8') as log:
    log.write(f'Se procesaron {len(ventas_ordenadas)} clientes en ventas.csv\n')

4. Lectura de clientes.xml
- Utilizar xml.etree.ElementTree para extraer:
        ◦ Nombre
        ◦ Edad
        ◦ Ciudad
- Vincular esta información con los nombres ya normalizados del paso 1.

In [4]:
import xml.etree.ElementTree as ET

tree = ET.parse('clientes.xml')
root = tree.getroot()

clientes_info = {}

for cliente in root.findall('cliente'):
    nombre = cliente.find('nombre').text.strip().title()
    edad = cliente.find('edad').text.strip()
    ciudad = cliente.find('ciudad').text.strip()
    if nombre in nombres_limpios:
        clientes_info[nombre] = {
            'edad': edad,
            'ciudad': ciudad
        }

with open('log.txt', 'a', encoding='utf-8') as log:
    log.write(f'Se vincularon {len(clientes_info)} clientes del XML con los nombres normalizados.\n')

5. Unificación de la información
- Para cada cliente presente en los archivos, combinar la información:
        ◦ Nombre limpio
        ◦ Ciudad (desde XML)
        ◦ Edad (desde XML)
        ◦ Total de ventas (desde CSV)
        ◦ Indicador es_mayor_edad (True si edad >= 18)
- Almacenar los datos combinados en una nueva estructura.

In [8]:
ventas_por_cliente = dict(zip(ventas_ordenadas['cliente'].str.strip().str.title(), ventas_ordenadas['monto_total']))

clientes_unificados = []

for nombre in nombres_limpios:
    ciudad = clientes_info.get(nombre, {}).get('ciudad')
    edad_str = clientes_info.get(nombre, {}).get('edad')
    edad = int(edad_str) if edad_str and edad_str.isdigit() else None
    total_ventas = ventas_por_cliente.get(nombre, 0)
    es_mayor_edad = edad is not None and edad >= 18

    cliente_dict = {
        'nombre': nombre,
        'ciudad': ciudad,
        'edad': edad,
        'total_ventas': total_ventas,
        'es_mayor_edad': es_mayor_edad
    }
    clientes_unificados.append(cliente_dict)

with open('log.txt', 'a', encoding='utf-8') as log:
    log.write(f'Se unificaron datos de {len(clientes_unificados)} clientes.\n')

6. Exportación y generación de resultados
- Exportar los datos finales a un archivo clientes_final.csv.
- Escribir en log.txt el mensaje:
        ◦ “Procesamiento finalizado con éxito.”
        ◦ Fecha y hora de ejecución
        ◦ Cantidad de registros procesados

In [10]:
import csv
from datetime import datetime

with open('clientes_final.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['nombre', 'ciudad', 'edad', 'total_ventas', 'es_mayor_edad']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for cliente in clientes_unificados:
        writer.writerow(cliente)

with open('log.txt', 'a', encoding='utf-8') as log:
    log.write('Procesamiento finalizado con éxito.\n')
    log.write(f'Fecha y hora de ejecución: {datetime.now()}\n')
    log.write(f'Cantidad de registros procesados: {len(clientes_unificados)}\n')